
# BigProtein-Qwen2.5 — Step‑by‑Step Test Notebook (Colab)
This notebook lets you **test each component** of the protein‑conditioned Qwen2.5 pipeline *before* running full training.  
It mirrors the main script logic, but runs **function‑by‑function** so you can see errors early with clear tracebacks.

> **Files expected in the working directory** (upload or mount a folder containing them):  
> - `bigmodel_joint_train.py`  
> - `protein_encoder.py`  
> - `structure_encoder.py`


In [1]:
#@title Mount Google Drive
from pathlib import Path
from huggingface_hub import snapshot_download
import os, json, pickle, pandas as pd
from tqdm import tqdm
from rich import print as rprint

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

%cd /content/drive/MyDrive/LLM/Bioreasoner/testing_pipelines

from pathlib import Path
BASE_DIR = Path("/content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT")
OUT_DIR  = BASE_DIR / "sft_test_demo"
print(f"Using Google Drive folder as BASE_DIR: {BASE_DIR}")


Mounted at /content/drive
/content/drive/MyDrive/LLM/Bioreasoner/testing_pipelines
Using Google Drive folder as BASE_DIR: /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT



## 0) Runtime & Installs
If you're on Google Colab, run this cell to install dependencies.


In [3]:
# Check GPU
!nvidia-smi

# Fresh pip + libs (PyTorch CUDA 12.1 build + matching libs)
%pip -q install --upgrade pip
%pip install -q --index-url https://download.pytorch.org/whl/cu126 \
  torch==2.8.0 torchvision==0.23.0 torchaudio==2.8.0
%pip -q install transformers==4.56.1 huggingface_hub==0.35.0 tqdm safetensors

Thu Oct  2 14:42:56 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off |   00000000:00:05.0 Off |                    0 |
| N/A   34C    P0             52W /  400W |       0MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [4]:
# --- Version & import sanity checks ---
import torch, transformers, huggingface_hub
print("torch            :", torch.__version__)
print("transformers     :", transformers.__version__)
print("huggingface_hub  :", huggingface_hub.__version__)

# Top-level ESM import should work on 4.56.1
try:
    from transformers import AutoTokenizer, EsmForMaskedLM
    print("✅ Top-level EsmForMaskedLM import OK")
except Exception as e:
    print("❌ Top-level EsmForMaskedLM import failed:", repr(e))
    # Fallback check (direct module path)
    try:
        from transformers.models.esm.modeling_esm import EsmForMaskedLM as _E
        print("✅ Direct modeling_esm import OK (fallback)")
    except Exception as ee:
        print("❌ Direct modeling_esm import failed too:", repr(ee))

torch            : 2.8.0+cu126
transformers     : 4.56.1
huggingface_hub  : 0.35.0
✅ Top-level EsmForMaskedLM import OK


torch            : 2.8.0+cu126

transformers     : 4.56.1

huggingface_hub  : 0.35.0

✅ Top-level EsmForMaskedLM import OK


## 1) Loading Encoder Checkpoints

In [5]:

# === LLM & Encoders ===
MODEL_NAME         = "Qwen/Qwen2.5-3B-Instruct"   # Small-ish for Colab testing
PROTEIN_CONFIG = "/content/drive/MyDrive/LLM/Bioreasoner/protrek/weights/ProTrek_35M/esm2_t12_35M_UR50D"
STRUCTURE_CONFIG = "/content/drive/MyDrive/LLM/Bioreasoner/protrek/weights/ProTrek_35M/foldseek_t12_35M"
PROTREK_CKPT    = "/content/drive/MyDrive/LLM/Bioreasoner/protrek/weights/ProTrek_35M/ProTrek_35M.pt"
PROJECT_DIR = "/content/drive/MyDrive/LLM/Bioreasoner/testing_pipelines"
DATA_JSONL = "/content/drive/MyDrive/LLM/Bioreasoner/testing_pipelines/protein2desc_sft_ALLFOUR_c000-009_fullcot.jsonl"
OUT_DIR = "/content/drive/MyDrive/LLM/Bioreasoner/testing_notebooks/runs_colab_test"

for p in [PROJECT_DIR, DATA_JSONL, PROTEIN_CONFIG, STRUCTURE_CONFIG, PROTREK_CKPT, OUT_DIR]:
    print("✓ exists:", os.path.exists(p), p)



✓ exists: True /content/drive/MyDrive/LLM/Bioreasoner/testing_pipelines
✓ exists: True /content/drive/MyDrive/LLM/Bioreasoner/testing_pipelines/protein2desc_sft_ALLFOUR_c000-009_fullcot.jsonl
✓ exists: True /content/drive/MyDrive/LLM/Bioreasoner/protrek/weights/ProTrek_35M/esm2_t12_35M_UR50D
✓ exists: True /content/drive/MyDrive/LLM/Bioreasoner/protrek/weights/ProTrek_35M/foldseek_t12_35M
✓ exists: True /content/drive/MyDrive/LLM/Bioreasoner/protrek/weights/ProTrek_35M/ProTrek_35M.pt
✓ exists: True /content/drive/MyDrive/LLM/Bioreasoner/testing_notebooks/runs_colab_test


In [6]:
# === Prefix/Proj ===
SINGLE_TOKEN_PREFIX = False     # True -> 1 token; False -> soft prefix of length PREFIX_LEN
PREFIX_LEN          = 4
PROJ_HID            = 1024
DROPOUT             = 0.10

# === Training toggles ===
USE_LORA            = False
TRAIN_ENCODERS      = False    # True = end-to-end; False = freeze encoders
FREEZE_PROTEIN      = False    # only used if TRAIN_ENCODERS=True
FREEZE_STRUCTURE    = False    # only used if TRAIN_ENCODERS=True
GRAD_CHECKPOINT     = False

# === Misc ===
DEVICE              = "cuda" if torch.cuda.is_available() else "cpu"
MAX_LEN             = 512
BSZ                 = 2
ACCUM               = 1
LR                  = 5e-5
WARMUP_RATIO        = 0.03
EPOCHS              = 1
OUTPUT_DIR          = "runs/colab_smoketest"
LOG_EVERY           = 1

print("Device:", DEVICE)

Device: cuda


In [8]:
# SUBSET_JSONL = os.path.join(PROJECT_DIR, "train_subset_100.jsonl")

# # Write first 1000 non-empty lines to subset
# count = 0
# with open(DATA_JSONL, "r", encoding="utf-8") as fin, open(SUBSET_JSONL, "w", encoding="utf-8") as fout:
#     for line in fin:
#         if not line.strip():
#             continue
#         fout.write(line)
#         count += 1
#         if count >= 100:
#             break

# print("Wrote subset lines:", count, "->", SUBSET_JSONL)

In [7]:
from train_prefix_qwen import train, parse_args
SUBSET_JSONL = os.path.join(PROJECT_DIR, "train_subset_100.jsonl")

In [ ]:
import types, os

SAVE_DIR = os.path.join(PROJECT_DIR, "runs_colab_test")

args = types.SimpleNamespace(
    # Data
    train_file   = DATA_JSONL,
    val_file     = None,
    batch_size   = 1,         # adjust if you want
    accum_steps  = 2,
    max_len      = 1024,       # keep modest for speed
    # Model
    model_name   = MODEL_NAME,
    dtype        = "fp32",    # or "bf16" on A100 for speed
    prefix_len   = 4,         # try 1 or 4+
    prefix_gate  = 0.5,       # stabilizer on the soft prefix
    learnable_gate = False,
    freeze_llm   = False,     # True = projector-only
    train_encoders = True,   # keep ESM encoders frozen for speed
    # Encoders
    protein_config = PROTEIN_CONFIG,
    structure_config = STRUCTURE_CONFIG,
    protrek_ckpt  = PROTREK_CKPT,
    prot_slot     = 1,
    stru_slot     = 3,
    # Optim
    epochs      = 3,
    lr          = 3e-5,       # projector+LLM small LR
    weight_decay= 0.05,
    # Save/eval
    save_dir    = OUT_DIR,
    save_every  = 1500,
    eval_every  = 0,
    # Misc
    seed        = 42,
)

# Kick off training
train(args)

Device: cuda


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Loaded LLM: Qwen/Qwen2.5-3B-Instruct | hidden_size=2048 | dtype=torch.float32
[ProteinEncoder] loaded from slot 1 | missing=0 unexpected=0
[StructureEncoder] loaded from slot 3 | missing=0 unexpected=0
[ep 1] step 50 | loss=3.0641 | supervised_tokens=36088 | time=1m16s
[ep 1] step 100 | loss=2.6345 | supervised_tokens=36838 | time=2m31s
[ep 1] step 150 | loss=2.6490 | supervised_tokens=37428 | time=3m46s
[ep 1] step 200 | loss=2.4567 | supervised_tokens=33908 | time=4m59s
[ep 1] step 250 | loss=2.4548 | supervised_tokens=38206 | time=6m15s
[ep 1] step 300 | loss=2.5506 | supervised_tokens=38324 | time=7m32s
[ep 1] step 350 | loss=2.3359 | supervised_tokens=35033 | time=8m45s
[ep 1] step 400 | loss=2.4895 | supervised_tokens=37708 | time=10m01s
[ep 1] step 450 | loss=2.1702 | supervised_tokens=36985 | time=11m16s
[ep 1] step 500 | loss=2.2701 | supervised_tokens=36464 | time=12m31s
[ep 1] step 550 | loss=1.9846 | supervised_tokens=35473 | time=13m45s
[ep 1] step 600 | loss=2.1731 | supe